In [2]:
sc.install_pypi_package('backtrader')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


The directory '/home/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
You are using pip version 9.0.1, however version 22.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import pyspark
# from ipywidgets import interact
import numpy as np
import pandas as pd
from tqdm import tqdm
import pandas_ta as ta

In [416]:
import requests
from tqdm import tqdm
SEC_URL = 'https://www.sec.gov/'
FSN_PATH = 'files/dera/data/financial-statement-and-notes-data-sets/'
filing_periods = [(d.year, d.quarter) for d in pd.date_range('2014', '2020-09-30', freq='Q')]
filing_periods

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[(2014, 1), (2014, 2), (2014, 3), (2014, 4), (2015, 1), (2015, 2), (2015, 3), (2015, 4), (2016, 1), (2016, 2), (2016, 3), (2016, 4), (2017, 1), (2017, 2), (2017, 3), (2017, 4), (2018, 1), (2018, 2), (2018, 3), (2018, 4), (2019, 1), (2019, 2), (2019, 3), (2019, 4), (2020, 1), (2020, 2), (2020, 3)]

In [ ]:
response = requests.get('https://www.sec.gov/files/dera/data/financial-statement-and-notes-data-sets/2020q3_notes.zip').content

In [ ]:
response

In [432]:
from pathlib import Path
from io import BytesIO
from zipfile import ZipFile, BadZipFile
from tqdm import tqdm
from pprint import pprint

# store data in this directory since we won't use it in other chapters
data_path = Path('data') # perhaps set to external harddrive to accomodate large amount of data
if not data_path.exists():
    data_path.mkdir()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
for yr, qtr in tqdm(filing_periods):
    # set (and create) directory
    path = data_path / f'{yr}_{qtr}' / 'source'
    if not path.exists():
        path.mkdir(parents=True)
    
    # define url and get file
    filing = f'{yr}q{qtr}_notes.zip'
    url = SEC_URL + FSN_PATH + filing
    response = requests.get(url).content
    
    # decompress and save
    try:
        with ZipFile(BytesIO(response)) as zip_file:
            for file in zip_file.namelist():
                local_file = path / file
                if local_file.exists():
                    continue
                with local_file.open('wb') as output:
                    for line in zip_file.open(file).readlines():
                        output.write(line)
    except BadZipFile:
        print(f'\nBad zip file: {yr} {qtr}\n')
        continue

In [ ]:
daily = spark.read.parquet('s3://algoseekraw/us_equity_daily_ohlc.parquet')

In [414]:
daily.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- TradeDate: date (nullable = true)
 |-- SecId: long (nullable = true)
 |-- Ticker: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- MarketHoursVolume: long (nullable = true)
 |-- CumulativePriceFactor: double (nullable = true)
 |-- CumulativeVolumeFactor: double (nullable = true)
 |-- AdjustmentFactor: double (nullable = true)
 |-- AdjustmentReason: string (nullable = true)

In [ ]:
daily = daily.withColumn('TradeDate', to_date(daily.TradeDate.cast(StringType()),"%YYYY%mm%dd"))

In [413]:
daily = daily.withColumn('

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df=spark.read.parquet("s3://strategymodeldata/equityprice/emrout/equity_1min_taq.parquet")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [237]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- TimeBarStart: string (nullable = true)
 |-- OpenBarTime: string (nullable = true)
 |-- OpenBidPrice: double (nullable = true)
 |-- OpenBidSize: integer (nullable = true)
 |-- OpenAskPrice: double (nullable = true)
 |-- OpenAskSize: integer (nullable = true)
 |-- FirstTradeTime: string (nullable = true)
 |-- FirstTradePrice: double (nullable = true)
 |-- FirstTradeSize: integer (nullable = true)
 |-- HighBidTime: string (nullable = true)
 |-- HighBidPrice: double (nullable = true)
 |-- HighBidSize: integer (nullable = true)
 |-- HighAskTime: string (nullable = true)
 |-- HighAskPrice: double (nullable = true)
 |-- HighAskSize: integer (nullable = true)
 |-- HighTradeTime: string (nullable = true)
 |-- HighTradePrice: double (nullable = true)
 |-- HighTradeSize: integer (nullable = true)
 |-- LowBidTime: string (nullable = true)
 |-- LowBidPrice: double (nullable = true)
 |-- LowBidSize: integer (nullable = true)
 |-- LowAskTime: string (nullable = true)
 |-- LowAskPrice: doubl

In [5]:
import datetime as dt
# Just a few days for testing
sDate = dt.datetime(2020, 1, 1)
eDate = dt.datetime(2020, 1, 7) 

df2 = ( df.filter( df.Date.between(sDate, eDate) ).filter(df.Ticker == "NFLX") )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
df2.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+------------------+------------+-----------+------------+-----------+------------------+---------------+--------------+------------------+------------+-----------+------------------+------------+-----------+------------------+--------------+-------------+------------------+-----------+----------+------------------+-----------+----------+------------------+-------------+------------+------------------+-------------+------------+-------------+------------+------------------+--------------+-------------+---------+---------+----------+-----------------+--------------+----------+-------------+----------+-------------+----------+--------------------+------+-----------+-----------+----------------------+------------+--------------+------------------+--------------------+-----------------+-------------------+---------------------------+-------------+-------------+-------------------+------+----------+
|TimeBarStart|       OpenBarTime|OpenBidPrice|OpenBidSize|OpenAskPrice|OpenAskS

In [253]:
# Filter the Spark DataFrame to be only one Ticker
testData = df.filter(df.Ticker == "MSFT") 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [316]:
multiTick = df.filter(df.Ticker.isin("MSFT","AAPL","AMZN","GOOG"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [207]:
tDat = testData.select('datetime',"FirstTradePrice",'HighTradePrice','LowTradePrice','LastTradePrice','Volume').collect()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [317]:
testData = testData.na.drop()
multiTick = multiTick.na.drop()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [255]:
import backtrader.feeds as btfeeds
import backtrader.analyzers as btanalyzers
from backtrader.feed import DataBase
from backtrader import date2num
from backtrader import TimeFrame
import os
import pytz
from pytz import timezone
import json
import itertools


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [285]:

class AlgoStrategy():
    
    def __init__(self,strategy):       
        self.cerebro = bt.Cerebro()        
        strategy.init_broker(self.cerebro.broker)
        
        data=strategy.add_data(self.cerebro)
        
        strategy.data=data
       
        self.cerebro.addstrategy(strategy)
        
        self.portfolioStartValue=self.cerebro.broker.getvalue()                            
        self.cerebro.addanalyzer(bt.analyzers.DrawDown, _name='dd')
        self.cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, _name='sharpe')
        self.cerebro.addanalyzer(bt.analyzers.SQN, _name='sqn')
        self.cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='ta')
        self.cerebro.addanalyzer(bt.analyzers.PyFolio, _name="pyfolio")
        
    def performance(self):
        analyzer=self.thestrat.analyzers.ta.get_analysis()
        dd_analyzer=self.thestrat.analyzers.dd.get_analysis()
      
        #Get the results we are interested in
        total_open = analyzer.total.open
        total_closed = analyzer.total.closed
        total_won = analyzer.won.total
        total_lost = analyzer.lost.total
        win_streak = analyzer.streak.won.longest
        lose_streak = analyzer.streak.lost.longest
        pnl_net = analyzer.pnl.net.total
        strike_rate=0
        if total_closed>0:
            strike_rate = (total_won / total_closed) * 100
        #Designate the rows
        h1 = ['Total Open', 'Total Closed', 'Total Won', 'Total Lost']
        h2 = ['Strike Rate','Win Streak', 'Losing Streak', 'PnL Net']
        h3 = ['DrawDown Pct','MoneyDown', '', '']
        self.total_closed=total_closed
        self.strike_rate=strike_rate
        self.max_drawdown=dd_analyzer.max.drawdown
        r1 = [total_open, total_closed,total_won,total_lost]
        r2 = [('%.2f%%' %(strike_rate)), win_streak, lose_streak, pnl_net]
        r3 = [('%.2f%%' %(dd_analyzer.max.drawdown)), dd_analyzer.max.moneydown, '', '']
        #Check which set of headers is the longest.
        # header_length = np.maximum(len(h1),len(h2),len(h3))
        #Print the rows
        print_list = [h1,r1,h2,r2,h3,r3]
        row_format ="{:<15}" * (200 + 1)
        print("Trade Analysis Results:")
        for row in print_list:
            #print(row_format.format('',*row))
            print(row)
        analyzer=self.thestrat.analyzers.sqn.get_analysis()
        sharpe_analyzer=self.thestrat.analyzers.sharpe.get_analysis()
        self.sqn = analyzer.sqn
        self.sharpe_ratio = sharpe_analyzer['sharperatio']
        if self.sharpe_ratio is None:
            self.sharpe_ratio=0
        self.pnl = self.cerebro.broker.getvalue()-self.portfolioStartValue
        print('[SQN:%.2f, Sharpe Ratio:%.2f, Final Portfolio:%.2f, Total PnL:%.2f]' % (self.sqn,self.sharpe_ratio,self.cerebro.broker.getvalue(),self.pnl))
        
    def run(self):
        thestrats = self.cerebro.run()
        self.thestrat = thestrats[0]
        self.performance()
        

class MyFeed(DataBase):
    def __init__(self):
        super(MyFeed, self).__init__()
        # self.list=testData.select("start", "open", "high", "low", "close", "volume", "vwap", "exponential_moving_average").collect()
        self.list=multiTick.select('datetime',"FirstTradePrice",'HighTradePrice','LowTradePrice','LastTradePrice','Volume','UptickVolume','DowntickVolume','RepeatUptickVolume','RepeatDowntickVolume').collect()
        self.n=0
        
        self.fromdate=self.list[0]['datetime']
        self.todate=self.list[len(self.list)-1]['datetime']
        self.timeframe=bt.TimeFrame.Minutes
        print("from=%s,to=%s" % (self.fromdate,self.todate))
        
        self.m={}
        
        #print(self.list)
        
    def start(self):
        # Nothing to do for this data feed type
        pass

    def stop(self):
        # Nothing to do for this data feed type
        pass
    
    def _load(self):
        if self.n>=len(self.list):
            return False
        
        r=self.list[self.n]
        self.lines.datetime[0] = date2num(r['datetime'])
        
        self.lines.open[0] = r['FirstTradePrice']
        self.lines.high[0] = r['HighTradePrice']
        self.lines.low[0] = r['LowTradePrice']
        self.lines.close[0] = r['LastTradePrice']
        self.lines.volume[0] = r['Volume']
        self.m[r['datetime']]=r
        
        self.n=self.n+1
        return True

class StrategyTemplate(bt.Strategy):
    
    def __init__(self):         
        self.lastDay=-1
        self.lastMonth=-1
        self.dataclose = self.datas[0].close
    
    @staticmethod
    def init_broker(broker):
        pass
    
    @staticmethod
    def add_data(cerebro):
        pass
     
    def next(self):
        dt=self.datas[0].datetime.datetime(0)
        print("[NEXT]:%s:close=%s" % (dt,self.dataclose[0]))
        
        #SOM
        if self.lastMonth!=dt.month:
            if self.lastMonth!=-1:
                chg=self.broker.getvalue()-self.monthCash
                print("[%s] SOM:chg=%.2f,cash=%.2f" % (dt,chg,self.broker.getvalue()))
            self.lastMonth=dt.month
            self.monthCash=self.broker.getvalue()
        
        #SOD
        if self.lastDay!=dt.day:
            self.lastDay=dt.day
            print("[%s] SOD:cash=%.2f" % (dt,self.broker.getvalue()))


tDat[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(datetime=datetime.datetime(2020, 1, 7, 4, 0), FirstTradePrice=336.97, HighTradePrice=336.97, LowTradePrice=336.97, LastTradePrice=336.97, Volume=10)

In [289]:

class MyStrategy(StrategyTemplate):

    def __init__(self):  # Initiation
        super(MyStrategy, self).__init__()
        
    def init_broker(broker):
        broker.setcash(1000000.0)
        broker.setcommission(commission=0.0) 
        
    def add_data(cerebro):
        data = MyFeed()
        
        cerebro.adddata(data)
        return data
    
    def next(self):  # Processing
        super(MyStrategy, self).next()
        dt=self.datas[0].datetime.datetime(0)
        r=self.data.m[dt]
        #print(r)
        size=self.cerebro.strat_params['size']
        threshold_PctChg=self.cerebro.strat_params['pct_chg']
               
        # model=self.cerebro.strat_params['model']
        df=spark.createDataFrame([r])
        # VWAP=r['vwap']
        # predictedVWAP = model.transform(df).collect()[0]['prediction']
        # expectedPctChg=(predictedVWAP-VWAP)/VWAP*100.0
        # print(self.data[0])
        
        # goLong=self.datas[0].close>self.datas[0].open
        # goShort=self.datas[0].close< self.datas[0].open
        goLong=r['UptickVolume']<r['DowntickVolume']
        goShort=r['UptickVolume']>r['DowntickVolume']
        # print("expectedPctChg=%s,goLong=%s,goShort=%s" % (expectedPctChg,goLong,goShort))
        
        if not self.position:
            if goLong:
                print("%s:%s x BUY @ %.2f" % (dt,size,r['LastTradePrice']))
                self.buy(size=size) # Go long
            else:
                print("%s:%s x SELL @ %.2f" % (dt,size,r['LastTradePrice']))
                self.sell(size=size) # Go short
        elif self.position.size>0 and goShort:
            print("%s:%s x SELL @ %.2f" % (dt,size*2,r['LastTradePrice']))
            self.sell(size=size*2)
        elif self.position.size<0 and goLong: 
            print("%s:%s x BUY @ %.2f" % (dt,size*2,r['LastTradePrice']))
            self.buy(size=size*2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [383]:
# TODO: Add code to dynamically fetch unique tickers - for now, I'm hardcoding
tickers_multi = ["MSFT", "AAPL", "AMZN","GOOG","NVDA"]

config = { "user" : "user",
  "fast_period" : "50",
  "slow_period" : "200",
  "size" : "100"
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [371]:
class MultiAlgoStrategy():
    
    def __init__(self,strategy):    
        self.cerebro = bt.Cerebro()        
        strategy.init_broker(self.cerebro.broker)
        
        data=strategy.add_data(self.cerebro,tick="MSFT")
        
        strategy.data=data
       
        self.cerebro.addstrategy(strategy)
        
        self.portfolioStartValue=self.cerebro.broker.getvalue()

        self.smaFast = bt.ind.SimpleMovingAverage(period=50)
        self.smaSlow = bt.ind.SimpleMovingAverage(period=200)
        
        self.cerebro.addanalyzer(bt.analyzers.DrawDown, _name='dd')
        self.cerebro.addanalyzer(bt.analyzers.SharpeRatio_A, _name='sharpe')
        self.cerebro.addanalyzer(bt.analyzers.SQN, _name='sqn')
        self.cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='ta')
        self.cerebro.addanalyzer(bt.analyzers.PyFolio, _name="pyfolio")
        
    def performance(self):
        analyzer=self.thestrat.analyzers.ta.get_analysis()
        dd_analyzer=self.thestrat.analyzers.dd.get_analysis()
      
        #Get the results we are interested in
        total_open = analyzer.total.open
        total_closed = analyzer.total.closed
        total_won = analyzer.won.total
        total_lost = analyzer.lost.total
        win_streak = analyzer.streak.won.longest
        lose_streak = analyzer.streak.lost.longest
        pnl_net = analyzer.pnl.net.total
        strike_rate=0
        if total_closed>0:
            strike_rate = (total_won / total_closed) * 100
        #Designate the rows
        h1 = ['Total Open', 'Total Closed', 'Total Won', 'Total Lost']
        h2 = ['Strike Rate','Win Streak', 'Losing Streak', 'PnL Net']
        h3 = ['DrawDown Pct','MoneyDown', '', '']
        self.total_closed=total_closed
        self.strike_rate=strike_rate
        self.max_drawdown=dd_analyzer.max.drawdown
        r1 = [total_open, total_closed,total_won,total_lost]
        r2 = [('%.2f%%' %(strike_rate)), win_streak, lose_streak, pnl_net]
        r3 = [('%.2f%%' %(dd_analyzer.max.drawdown)), dd_analyzer.max.moneydown, '', '']
        #Check which set of headers is the longest.
        # header_length = np.maximum(len(h1),len(h2),len(h3))
        #Print the rows
        print_list = [h1,r1,h2,r2,h3,r3]
        row_format ="{:<15}" * (200 + 1)
        print("Trade Analysis Results:")
        for row in print_list:
            #print(row_format.format('',*row))
            print(row)
        analyzer=self.thestrat.analyzers.sqn.get_analysis()
        sharpe_analyzer=self.thestrat.analyzers.sharpe.get_analysis()
        self.sqn = analyzer.sqn
        self.sharpe_ratio = sharpe_analyzer['sharperatio']
        if self.sharpe_ratio is None:
            self.sharpe_ratio=0
        self.pnl = self.cerebro.broker.getvalue()-self.portfolioStartValue
        print('[SQN:%.2f, Sharpe Ratio:%.2f, Final Portfolio:%.2f, Total PnL:%.2f]' % (self.sqn,self.sharpe_ratio,self.cerebro.broker.getvalue(),self.pnl))
        
    def run(self):
        thestrats = self.cerebro.run()
        self.thestrat = thestrats[0]
        self.performance()
        
class MultiFeed(DataBase):
    def __init__(self):
        super(MultiFeed, self).__init__()
        # self.list=testData.select("start", "open", "high", "low", "close", "volume", "vwap", "exponential_moving_average").collect()
        
        self.list=multiTick.filter(multiTick.Ticker==self.p.dataname).select('datetime',"FirstTradePrice",'HighTradePrice','LowTradePrice','LastTradePrice','Volume','UptickVolume','DowntickVolume','RepeatUptickVolume','RepeatDowntickVolume').collect()
        self.n=0
        self.fromdate=self.list[0]['datetime']
        self.todate=self.list[len(self.list)-1]['datetime']
        self.timeframe=bt.TimeFrame.Minutes
        print("from=%s,to=%s" % (self.fromdate,self.todate))
        
        self.m={}
        
        #print(self.list)
        
    def start(self):
        # Nothing to do for this data feed type
        pass

    def stop(self):
        # Nothing to do for this data feed type
        pass
    
    def _load(self):
        if self.n>=len(self.list):
            return False
        
        r=self.list[self.n]
        self.lines.datetime[0] = date2num(r['datetime'])
        
        self.lines.open[0] = r['FirstTradePrice']
        self.lines.high[0] = r['HighTradePrice']
        self.lines.low[0] = r['LowTradePrice']
        self.lines.close[0] = r['LastTradePrice']
        self.lines.volume[0] = r['Volume']
        self.m[r['datetime']]=r
        
        self.n=self.n+1
        return True

class MultiTickStrategy(StrategyTemplate):

    def __init__(self):  # Initiation
        super(MultiTickStrategy, self).__init__()
        self
        self.config["fast_period"]=int(self.config["fast_period"])
        self.config["slow_period"]=int(self.config["slow_period"])
        self.config["size"]=int(self.config["size"])

        self.smaFast = bt.ind.SimpleMovingAverage(period=self.config["fast_period"])
        self.smaSlow = bt.ind.SimpleMovingAverage(period=self.config["slow_period"])
        self.size = self.config["size"]
        
    def init_broker(broker):
        broker.setcash(1000000.0)
        broker.setcommission(commission=0.0) 
        
    def add_data(cerebro,tick):
        # TODO: Parameterize the code
        data = MultiFeed(dataname=tick)
        
        cerebro.adddata(data,name=tick)
        return data
    
    def next(self):  # Processing
        super(MultiTickStrategy, self).next()
        dt=self.datas[0].datetime.datetime(0)
        r=self.data.m[dt]
        #print(r)
        size=self.cerebro.strat_params['size']
        threshold_PctChg=self.cerebro.strat_params['pct_chg']
               
        # model=self.cerebro.strat_params['model']
        df=spark.createDataFrame([r])
        # VWAP=r['vwap']
        # predictedVWAP = model.transform(df).collect()[0]['prediction']
        # expectedPctChg=(predictedVWAP-VWAP)/VWAP*100.0
        # print(self.data[0])
        
        postitions=[d._name for d, pos in self.getpositions().items() if pos]
        
        # goLong=self.datas[0].close>self.datas[0].open
        # goShort=self.datas[0].close< self.datas[0].open
        goLong=r['UptickVolume']<r['DowntickVolume']
        goShort=r['UptickVolume']>r['DowntickVolume']
        # print("expectedPctChg=%s,goLong=%s,goShort=%s" % (expectedPctChg,goLong,goShort))
        
        if not self.position:
            if goLong:
                print("%s:%s x BUY @ %.2f" % (dt,size,r['LastTradePrice']))
                self.buy(size=size) # Go long
            else:
                print("%s:%s x SELL @ %.2f" % (dt,size,r['LastTradePrice']))
                self.sell(size=size) # Go short
        elif self.position.size>0 and goShort:
            print("%s:%s x SELL @ %.2f" % (dt,size*2,r['LastTradePrice']))
            self.sell(size=size*2)
        elif self.position.size<0 and goLong: 
            print("%s:%s x BUY @ %.2f" % (dt,size*2,r['LastTradePrice']))
            self.buy(size=size*2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [372]:
scenarios=[]
for p in range(0,5):
    for s in range(0,1):
        c={'scenario':(p+1), "size":100,"pct_chg":0.01}
        print(c)
    scenarios.append(c)
scenarios

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'scenario': 1, 'size': 100, 'pct_chg': 0.01}
{'scenario': 2, 'size': 100, 'pct_chg': 0.01}
{'scenario': 3, 'size': 100, 'pct_chg': 0.01}
{'scenario': 4, 'size': 100, 'pct_chg': 0.01}
{'scenario': 5, 'size': 100, 'pct_chg': 0.01}
[{'scenario': 1, 'size': 100, 'pct_chg': 0.01}, {'scenario': 2, 'size': 100, 'pct_chg': 0.01}, {'scenario': 3, 'size': 100, 'pct_chg': 0.01}, {'scenario': 4, 'size': 100, 'pct_chg': 0.01}, {'scenario': 5, 'size': 100, 'pct_chg': 0.01}]

In [373]:
# run scenarios
best_config=None
best_pnl=None
n=0
for c in scenarios:
    print("*** [%s] RUN SCENARIO:%s" % ((n+1),c))
    config=c
    algo=MultiAlgoStrategy(MultiTickStrategy)
    algo.cerebro.strat_params=config
    algo.run()
    ending_value = algo.cerebro.broker.getvalue()
    if best_pnl is None or best_pnl<ending_value:
        best_config=c
        best_pnl=ending_value
    n+=1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

*** [1] RUN SCENARIO:{'scenario': 1, 'size': 100, 'pct_chg': 0.01}
from=2021-10-27 10:14:00,to=2019-06-12 15:24:00
[NEXT]:2021-10-27 10:14:00:close=319.285
[2021-10-27 10:14:00] SOD:cash=1000000.00
2021-10-27 10:14:00:100 x BUY @ 319.29
[NEXT]:2021-10-27 14:49:00:close=325.675
[NEXT]:2020-04-29 10:12:00:close=173.77
[2020-04-29 10:12:00] SOM:chg=-15197.00,cash=984803.00
[2020-04-29 10:12:00] SOD:cash=984803.00
2020-04-29 10:12:00:200 x SELL @ 173.77
[NEXT]:2020-04-29 15:00:00:close=176.33
2020-04-29 15:00:00:200 x BUY @ 176.33
[NEXT]:2020-04-29 15:28:00:close=177.395
[NEXT]:2020-04-29 16:49:00:close=185.1
2020-04-29 16:49:00:200 x SELL @ 185.10
[NEXT]:2018-10-26 15:25:00:close=106.5789
[2018-10-26 15:25:00] SOM:chg=-6935.11,cash=977867.89
[2018-10-26 15:25:00] SOD:cash=977867.89
[NEXT]:2018-10-26 15:39:00:close=106.599
[NEXT]:2018-10-26 15:25:00:close=106.5789
[NEXT]:2018-10-26 15:39:00:close=106.599
[NEXT]:2020-03-19 12:11:00:close=145.3602
[2020-03-19 12:11:00] SOM:chg=3878.13,cash=9

## Test using fundamentals

In [293]:
import awswrangler as wr
import requests

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
headers = {'User-Agent': "julian@julianwiley.com"}

tickers_cik = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers)
raw_cik = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers)
# raw_cik.json()

In [295]:
tickers_cik = pd.json_normalize(pd.json_normalize(tickers_cik.json(),max_level=0).values[0])
tickers_cik["cik_str"] = tickers_cik["cik_str"].astype(str).str.zfill(10)
tickers_cik.set_index("ticker",inplace=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [299]:
tickers_cik.loc['MSFT']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

cik_str        0000789019
title      MICROSOFT CORP
Name: MSFT, dtype: object

In [391]:
msft_facts='s3://securityfundamentalsdata/companyfacts/CIK'+tickers_cik.loc['MSFT']['cik_str']+'.json'
msft_json = wr.s3.read_json(msft_facts)
msft_spark = spark.read.option("multiline","true").json(msft_facts)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [397]:
msft_spark= msft_spark.withColumn('gaap',col("facts.us-gaap")).withColumn('dei',col("facts.dei")).drop("facts")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [398]:
msft_spark.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cik: long (nullable = true)
 |-- entityName: string (nullable = true)
 |-- gaap: struct (nullable = true)
 |    |-- AccountsPayableCurrent: struct (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- label: string (nullable = true)
 |    |    |-- units: struct (nullable = true)
 |    |    |    |-- USD: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- accn: string (nullable = true)
 |    |    |    |    |    |-- end: string (nullable = true)
 |    |    |    |    |    |-- filed: string (nullable = true)
 |    |    |    |    |    |-- form: string (nullable = true)
 |    |    |    |    |    |-- fp: string (nullable = true)
 |    |    |    |    |    |-- frame: string (nullable = true)
 |    |    |    |    |    |-- fy: long (nullable = true)
 |    |    |    |    |    |-- val: long (nullable = true)
 |    |-- AccountsReceivableNet: struct (nullable = true)
 |    |    |-- description: 

In [405]:
msft_spark.select("gaap.EarningsPerShareBasic.units.USD/shares")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[USD/shares: array<struct<accn:string,end:string,filed:string,form:string,fp:string,frame:string,fy:bigint,start:string,val:double>>]

In [385]:
facts = msft_spark.select('facts')
dei = facts.select('dei')
gaap = facts.select('us-gaap')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
cannot resolve 'dei' given input columns: [facts];
'Project ['dei]
+- Project [facts#1580124]
   +- Relation [cik#1580122L,entityName#1580123,facts#1580124] json

Traceback (most recent call last):
  File "/mnt/yarn/usercache/livy/appcache/application_1653363458253_0001/container_1653363458253_0001_01_000001/pyspark.zip/pyspark/sql/dataframe.py", line 1686, in select
    jdf = self._jdf.select(self._jcols(*cols))
  File "/mnt/yarn/usercache/livy/appcache/application_1653363458253_0001/container_1653363458253_0001_01_000001/py4j-0.10.9.2-src.zip/py4j/java_gateway.py", line 1310, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/mnt/yarn/usercache/livy/appcache/application_1653363458253_0001/container_1653363458253_0001_01_000001/pyspark.zip/pyspark/sql/utils.py", line 117, in deco
    raise converted from None
pyspark.sql.utils.AnalysisException: cannot resolve 'dei' given input columns: [facts];
'Project ['dei]
+- Project [facts#

In [401]:
fact2 = spark.read.option("multiline","true").json('s3://securityfundamentalsdata/companyfacts/**.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Found duplicate column(s) in the data schema: `employee`, `employees`, `item`, `number`, `person`
Traceback (most recent call last):
  File "/mnt/yarn/usercache/livy/appcache/application_1653363458253_0001/container_1653363458253_0001_01_000001/pyspark.zip/pyspark/sql/readwriter.py", line 229, in json
    return self._df(self._jreader.json(self._spark._sc._jvm.PythonUtils.toSeq(path)))
  File "/mnt/yarn/usercache/livy/appcache/application_1653363458253_0001/container_1653363458253_0001_01_000001/py4j-0.10.9.2-src.zip/py4j/java_gateway.py", line 1310, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/mnt/yarn/usercache/livy/appcache/application_1653363458253_0001/container_1653363458253_0001_01_000001/pyspark.zip/pyspark/sql/utils.py", line 117, in deco
    raise converted from None
pyspark.sql.utils.AnalysisException: Found duplicate column(s) in the data schema: `employee`, `employees`, `item`, `number`, `person`



In [390]:
f

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
cannot resolve 'dei' given input columns: [facts];
'Project ['dei]
+- Project [facts#1580124]
   +- Project [facts#1580124]
      +- Relation [cik#1580122L,entityName#1580123,facts#1580124] json

Traceback (most recent call last):
  File "/mnt/yarn/usercache/livy/appcache/application_1653363458253_0001/container_1653363458253_0001_01_000001/pyspark.zip/pyspark/sql/dataframe.py", line 1686, in select
    jdf = self._jdf.select(self._jcols(*cols))
  File "/mnt/yarn/usercache/livy/appcache/application_1653363458253_0001/container_1653363458253_0001_01_000001/py4j-0.10.9.2-src.zip/py4j/java_gateway.py", line 1310, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/mnt/yarn/usercache/livy/appcache/application_1653363458253_0001/container_1653363458253_0001_01_000001/pyspark.zip/pyspark/sql/utils.py", line 117, in deco
    raise converted from None
pyspark.sql.utils.AnalysisException: cannot resolve 'dei' given input columns: [facts];
'

In [434]:
msft_spark.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- cik: long (nullable = true)
 |-- entityName: string (nullable = true)
 |-- gaap: struct (nullable = true)
 |    |-- AccountsPayableCurrent: struct (nullable = true)
 |    |    |-- description: string (nullable = true)
 |    |    |-- label: string (nullable = true)
 |    |    |-- units: struct (nullable = true)
 |    |    |    |-- USD: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- accn: string (nullable = true)
 |    |    |    |    |    |-- end: string (nullable = true)
 |    |    |    |    |    |-- filed: string (nullable = true)
 |    |    |    |    |    |-- form: string (nullable = true)
 |    |    |    |    |    |-- fp: string (nullable = true)
 |    |    |    |    |    |-- frame: string (nullable = true)
 |    |    |    |    |    |-- fy: long (nullable = true)
 |    |    |    |    |    |-- val: long (nullable = true)
 |    |-- AccountsReceivableNet: struct (nullable = true)
 |    |    |-- description: 

In [438]:
msft_spark.select('gaap')

VBox()

An error was encountered:
Session 0 unexpectedly reached final status 'dead'. See logs:
stdout: 

stderr: 
22/05/24 04:08:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/24 04:08:19 INFO RMProxy: Connecting to ResourceManager at ip-10-0-20-90.ec2.internal/10.0.20.90:8032
22/05/24 04:08:19 INFO Client: Requesting a new application from cluster with 4 NodeManagers
22/05/24 04:08:19 INFO Configuration: resource-types.xml not found
22/05/24 04:08:19 INFO ResourceUtils: Unable to find 'resource-types.xml'.
22/05/24 04:08:19 INFO Client: Verifying our application has not requested more than the maximum memory capability of the cluster (12288 MB per container)
22/05/24 04:08:19 INFO Client: Will allocate AM container, with 2432 MB memory including 384 MB overhead
22/05/24 04:08:19 INFO Client: Setting up container launch context for our AM
22/05/24 04:08:19 INFO Client: Setting up the launch environment for 

In [312]:
msft = msft_json.loc['us-gaap']['facts']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [315]:
pd.json_normalize(msft['Assets']['units']['USD'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

            end           val  ...       filed      frame
0    2009-06-30   77888000000  ...  2009-10-23        NaN
1    2009-06-30   77888000000  ...  2010-01-28        NaN
2    2009-06-30   77888000000  ...  2010-04-22        NaN
3    2009-06-30   77888000000  ...  2010-07-30  CY2009Q2I
4    2009-09-30   81612000000  ...  2009-10-23  CY2009Q3I
..          ...           ...  ...         ...        ...
101  2021-06-30  333779000000  ...  2022-01-25        NaN
102  2021-06-30  333779000000  ...  2022-04-26  CY2021Q2I
103  2021-09-30  335418000000  ...  2021-10-26  CY2021Q3I
104  2021-12-31  340389000000  ...  2022-01-25  CY2021Q4I
105  2022-03-31  344607000000  ...  2022-04-26  CY2022Q1I

[106 rows x 8 columns]

In [435]:
response = requests.get("https://www.sec.gov/files/dera/data/financial-statement-and-notes-data-sets/2022_04_notes.zip",headers=headers).content

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [436]:
response

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
Invalid status code '400' from http://localhost:8998/sessions/0/statements/436 with error payload: {"msg":"requirement failed: Session isn't active."}


## Pure Python Code from Ml4trading

In [204]:
dfPandas = df2.toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [199]:
dfPandas.set_index(['Ticker','datetime'],inplace=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
dfPandas.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                           TimeBarStart  ...        Date
Ticker datetime                          ...            
NFLX   2020-01-07 04:00:00        04:00  ...  2020-01-07
       2020-01-07 04:01:00        04:01  ...  2020-01-07
       2020-01-07 04:02:00        04:02  ...  2020-01-07
       2020-01-07 04:03:00        04:03  ...  2020-01-07
       2020-01-07 04:04:00        04:04  ...  2020-01-07

[5 rows x 60 columns]

In [52]:


columns = {'volumeweightprice'          : 'price',
           'finravolume'                : 'fvolume',
           'finravolumeweightprice'     : 'fprice',
           'uptickvolume'               : 'up',
           'downtickvolume'             : 'down',
           'repeatuptickvolume'         : 'rup',
           'repeatdowntickvolume'       : 'rdown',
           'firsttradeprice'            : 'open',
           'hightradeprice'             : 'high',
           'lowtradeprice'              : 'low',
           'lasttradeprice'             : 'close',
           'nbboquotecount'             : 'nbbo',
           'totaltrades'                : 'ntrades',
           'openbidprice'               : 'obprice',
           'openbidsize'                : 'obsize',
           'openaskprice'               : 'oaprice',
           'openasksize'                : 'oasize',
           'highbidprice'               : 'hbprice',
           'highbidsize'                : 'hbsize',
           'highaskprice'               : 'haprice',
           'highasksize'                : 'hasize',
           'lowbidprice'                : 'lbprice',
           'lowbidsize'                 : 'lbsize',
           'lowaskprice'                : 'laprice',
           'lowasksize'                 : 'lasize',
           'closebidprice'              : 'cbprice',
           'closebidsize'               : 'cbsize',
           'closeaskprice'              : 'caprice',
           'closeasksize'               : 'casize',
           'firsttradesize'             : 'firstsize',
           'hightradesize'              : 'highsize',
           'lowtradesize'               : 'lowsize',
           'lasttradesize'              : 'lastsize',
           'tradetomidvolweight'        : 'volweight',
           'tradetomidvolweightrelative': 'volweightrel'}



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
dfPandas=dfPandas.rename(columns=str.lower)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
dfPandas=dfPandas.rename(columns=columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
import backtrader as bt
from backtrader.feeds import PandasData

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
class FixedCommisionScheme(bt.CommInfoBase):
    """
    Simple fixed commission scheme for demo
    """
    params = (
        ('commission', .02),
        ('stocklike', True),
        ('commtype', bt.CommInfoBase.COMM_FIXED),
    )

    def _getcommission(self, size, price, pseudoexec):
        return abs(size) * self.p.commission

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
OHLCV = ['open', 'high','low','close','volume']
ohlcv_cols = ['open', 'high', 'low', 'close', 'price', 'volume']
data_cols = ohlcv_cols + ['up', 'down', 'rup', 'rdown']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [87]:
dfP = dfPandas.loc[:,data_cols]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [88]:
dfP

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                              open    high     low  ...  down  rup  rdown
Ticker datetime                                     ...                  
NFLX   2020-01-07 04:00:00  336.97  336.97  336.97  ...     0    0      0
       2020-01-07 04:01:00     NaN     NaN     NaN  ...     0    0      0
       2020-01-07 04:02:00  336.50  336.50  336.50  ...   100    0      0
       2020-01-07 04:03:00     NaN     NaN     NaN  ...     0    0      0
       2020-01-07 04:04:00     NaN     NaN     NaN  ...     0    0      0
...                            ...     ...     ...  ...   ...  ...    ...
       2020-01-03 19:55:00  326.15  326.15  326.15  ...     0    0      0
       2020-01-03 19:56:00     NaN     NaN     NaN  ...     0    0      0
       2020-01-03 19:57:00     NaN     NaN     NaN  ...     0    0      0
       2020-01-03 19:58:00  326.15  326.25  326.15  ...     0   55      0
       2020-01-03 19:59:00  326.26  326.29  326.26  ...     0    0      0

[3840 rows x 10 columns]

In [86]:
dfPandas.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Index(['timebarstart', 'openbartime', 'obprice', 'obsize', 'oaprice', 'oasize',
       'firsttradetime', 'open', 'firstsize', 'highbidtime', 'hbprice',
       'hbsize', 'highasktime', 'haprice', 'hasize', 'hightradetime', 'high',
       'highsize', 'lowbidtime', 'lbprice', 'lbsize', 'lowasktime', 'laprice',
       'lasize', 'lowtradetime', 'low', 'lowsize', 'closebartime', 'cbprice',
       'cbsize', 'caprice', 'casize', 'lasttradetime', 'close', 'lastsize',
       'minspread', 'maxspread', 'cancelsize', 'price', 'nbbo', 'tradeatbid',
       'tradeatbidmid', 'tradeatmid', 'tradeatmidask', 'tradeatask',
       'tradeatcrossorlocked', 'volume', 'ntrades', 'fvolume', 'fprice', 'up',
       'down', 'rup', 'rdown', 'unknowntickvolume', 'volweight',
       'volweightrel', 'timeweightbid', 'timeweightask', 'date'],
      dtype='object')

In [66]:


DataCols = [
    'TimeBarStart',
    'OpenBarTime',
    'OpenBidPrice',
    'OpenBidSize',
    'OpenAskPrice',
    'OpenAskSize','FirstTradeTime',
    'FirstTradePrice','FirstTradeSize',
    'HighBidTime','HighBidPrice','HighBidSize',
    'HighAskTime','HighAskPrice','HighAskSize',
    'HighTradeTime','HighTradePrice',
    'HighTradeSize',
    'LowBidTime',
    'LowBidPrice',
    'LowBidSize',
    'LowAskTime','LowAskPrice','LowAskSize',
    'LowTradeTime','LowTradePrice',
    'LowTradeSize','CloseBarTime',
    'CloseBidPrice','CloseBidSize',
    'CloseAskPrice','CloseAskSize',
    'LastTradeTime',
    'LastTradePrice',
    'LastTradeSize','MinSpread',
    'MaxSpread','CancelSize',
    'VolumeWeightPrice','NBBOQuoteCount',
    'TradeAtBid','TradeAtBidMid','TradeAtMid',
    'TradeAtMidAsk','TradeAtAsk',
    'TradeAtCrossOrLocked',
    'Volume','TotalTrades','FinraVolume',
    'FinraVolumeWeightPrice','UptickVolume',
    'DowntickVolume','RepeatUptickVolume',
    'RepeatDowntickVolume',
    'UnknownTickVolume','TradeToMidVolWeight',
    'TradeToMidVolWeightRelative',
    'TimeWeightBid',
    'TimeWeightAsk']
dfCols=['timebarstart', 'openbartime', 'obprice', 'obsize', 'oaprice', 'oasize',
       'firsttradetime', 'open', 'firstsize', 'highbidtime', 'hbprice',
       'hbsize', 'highasktime', 'haprice', 'hasize', 'hightradetime', 'high',
       'highsize', 'lowbidtime', 'lbprice', 'lbsize', 'lowasktime', 'laprice',
       'lasize', 'lowtradetime', 'low', 'lowsize', 'closebartime', 'cbprice',
       'cbsize', 'caprice', 'casize', 'lasttradetime', 'close', 'lastsize',
       'minspread', 'maxspread', 'cancelsize', 'price', 'nbbo', 'tradeatbid',
       'tradeatbidmid', 'tradeatmid', 'tradeatmidask', 'tradeatask',
       'tradeatcrossorlocked', 'volume', 'ntrades', 'fvolume', 'fprice', 'up',
       'down', 'rup', 'rdown', 'unknowntickvolume', 'volweight',
       'volweightrel', 'timeweightbid', 'timeweightask','date']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [67]:
class FullAlgoSeekData(PandasData):
    """
    Define pandas DataFrame structure
    """
    cols = dfCols

    # create lines
    lines = tuple(cols)

    # define parameters
    params = {c: -1 for c in cols}
    params.update({'datetime': None})
    params.update({'timeframe':bt.TimeFrame.Minutes})
    params = tuple(params.items())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [169]:
class MLStrategy(bt.Strategy):
    params = (('n_positions', 10),
              ('min_positions', 5),
              ('verbose', False),
              ('log_file', 'backtest.csv'))

    def log(self, txt, dt=None):
        """ Logger for the strategy"""
        dt = dt or self.datas[0].datetime.datetime(0)
        with Path(self.p.log_file).open('a') as f:
            log_writer = csv.writer(f)
            log_writer.writerow([dt.isoformat()] + txt.split(','))

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        # Check if an order has been completed
        # broker could reject order if not enough cash
        if self.p.verbose:
            if order.status in [order.Completed]:
                p = order.executed.price
                if order.isbuy():
                    self.log(f'{order.data._name},BUY executed,{p:.2f}')
                    print(f'{order.data._name},BUY executed,{p:.2f}')

                elif order.issell():
                    self.log(f'{order.data._name},SELL executed,{p:.2f}')
                    print(f'{order.data._name},SELL executed,{p:.2f}')

            elif order.status in [order.Canceled, order.Margin, order.Rejected]:
                self.log(f'{order.data._name},Order Canceled/Margin/Rejected')
                print(f'{order.data._name},Order Canceled/Margin/Rejected')

    # bt calls prenext instead of next unless
    # all datafeeds have current values
    # => call next to avoid duplicating logic
    def prenext(self):
        self.next()

    def next(self):
        today = self.datas[0].datetime.date()
#         if today.weekday() not in [0, 3]: # only trade on Mondays;
#             return
        positions = [d._name for d, pos in self.getpositions().items() if pos]
        up, down = {}, {}
        missing = not_missing = 0
        for data in self.datas:
            if data.datetime.date() == today:
                if data.close[0] > data.open[0]:
                    up[data._name] = data.close[0]
                elif data.close[0] < data.open[0]:
                    down[data._name] = data.close[0]

        # sort dictionaries ascending/descending by value
        # returns list of tuples
        
        shorts = sorted(down, key=down.get)[:self.p.n_positions]
        longs = sorted(up, key=up.get, reverse=True)[:self.p.n_positions]
        n_shorts, n_longs = len(shorts), len(longs)
        
        # only take positions if at least min_n longs and shorts
        if n_shorts < self.p.min_positions or n_longs < self.p.min_positions:
            longs, shorts = [], []
        for ticker in positions:
            if ticker not in longs + shorts:
                self.order_target_percent(data=ticker, target=0)
                self.log(f'{ticker},CLOSING ORDER CREATED')
                print(f'{ticker},CLOSING ORDER CREATED')

        short_target = -1 / np.maximum(self.p.n_positions, n_shorts)
        long_target = 1 / np.maximum(self.p.n_positions, n_longs)
        for ticker in shorts:
            self.order_target_percent(data=ticker, target=short_target)
            self.log('{ticker},SHORT ORDER CREATED')
            print('{ticker},SHORT ORDER CREATED')

        for ticker in longs:
            self.order_target_percent(data=ticker, target=long_target)
            self.log('{ticker},LONG ORDER CREATED')
            print('{ticker},LONG ORDER CREATED')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [170]:
import numpy as np

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [171]:
cerebro = bt.Cerebro()  # create a "Cerebro" instance
cash = 10000
# comminfo = FixedCommisionScheme()
# cerebro.broker.addcommissioninfo(comminfo)
cerebro.broker.setcash(cash)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [172]:
idx = pd.IndexSlice
#data = pd.read_hdf('00_data/backtest.h5', 'data').sort_index()
dfP = dfP.dropna().sort_index()
tickers = dfP.index.get_level_values('Ticker').unique()

for ticker in tickers:
    df3 = dfP.loc[idx[ticker, :], :].droplevel('Ticker', axis=0)
    df3.index.name = 'datetime'
    bt_data = FullAlgoSeekData(dataname=df3)
    cerebro.adddata(bt_data, name=ticker)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [173]:
dfP.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                              open    high     low  ...  down  rup  rdown
Ticker datetime                                     ...                  
NFLX   2020-01-02 04:00:00  324.50  324.50  324.50  ...     0    0      0
       2020-01-02 04:30:00  326.12  326.12  326.00  ...    10    0     50
       2020-01-02 04:34:00  326.12  326.12  326.12  ...     0    5      0
       2020-01-02 04:55:00  326.12  326.12  326.12  ...     0   81      0
       2020-01-02 04:59:00  326.12  326.12  326.12  ...     0  300      0

[5 rows x 10 columns]

In [174]:
df3.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                       open    high     low   close  ...  up  down  rup  rdown
datetime                                             ...                      
2020-01-02 04:00:00  324.50  324.50  324.50  324.50  ...   0     0    0      0
2020-01-02 04:30:00  326.12  326.12  326.00  326.12  ...  40    10    0     50
2020-01-02 04:34:00  326.12  326.12  326.12  326.12  ...   0     0    5      0
2020-01-02 04:55:00  326.12  326.12  326.12  326.12  ...   0     0   81      0
2020-01-02 04:59:00  326.12  326.12  326.12  326.12  ...   0     0  300      0

[5 rows x 10 columns]

In [175]:
df3.info

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<bound method DataFrame.info of                        open     high     low   close  ...   up  down  rup  rdown
datetime                                              ...                       
2020-01-02 04:00:00  324.50  324.500  324.50  324.50  ...    0     0    0      0
2020-01-02 04:30:00  326.12  326.120  326.00  326.12  ...   40    10    0     50
2020-01-02 04:34:00  326.12  326.120  326.12  326.12  ...    0     0    5      0
2020-01-02 04:55:00  326.12  326.120  326.12  326.12  ...    0     0   81      0
2020-01-02 04:59:00  326.12  326.120  326.12  326.12  ...    0     0  300      0
...                     ...      ...     ...     ...  ...  ...   ...  ...    ...
2020-01-07 19:55:00  323.00  323.000  322.25  323.00  ...  135    72  201     38
2020-01-07 19:56:00  323.00  323.045  323.00  323.00  ...   15    10   50     22
2020-01-07 19:57:00  323.00  323.000  322.10  322.10  ...   81    74    0    256
2020-01-07 19:58:00  322.10  323.000  322.10  322.99  ...   50    10    5    

In [176]:
df3.dropna().sort_index()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                       open     high     low   close  ...   up  down  rup  rdown
datetime                                              ...                       
2020-01-02 04:00:00  324.50  324.500  324.50  324.50  ...    0     0    0      0
2020-01-02 04:30:00  326.12  326.120  326.00  326.12  ...   40    10    0     50
2020-01-02 04:34:00  326.12  326.120  326.12  326.12  ...    0     0    5      0
2020-01-02 04:55:00  326.12  326.120  326.12  326.12  ...    0     0   81      0
2020-01-02 04:59:00  326.12  326.120  326.12  326.12  ...    0     0  300      0
...                     ...      ...     ...     ...  ...  ...   ...  ...    ...
2020-01-07 19:55:00  323.00  323.000  322.25  323.00  ...  135    72  201     38
2020-01-07 19:56:00  323.00  323.045  323.00  323.00  ...   15    10   50     22
2020-01-07 19:57:00  323.00  323.000  322.10  322.10  ...   81    74    0    256
2020-01-07 19:58:00  322.10  323.000  322.10  322.99  ...   50    10    5    155
2020-01-07 19:59:00  323.00 

In [177]:
from time import time
def format_time(t):
    m_, s = divmod(t, 60)
    h, m = divmod(m_, 60)
    return f'{h:>02.0f}:{m:>02.0f}:{s:>02.0f}'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [178]:
cerebro.addanalyzer(bt.analyzers.PyFolio, _name='pyfolio')
cerebro.addstrategy(MLStrategy, n_positions=25, min_positions=20, 
                    verbose=True, log_file='bt_log.csv')
start = time()
results = cerebro.run()
ending_value = cerebro.broker.getvalue()
duration = time() - start

print(f'Final Portfolio Value: {ending_value:,.2f}')
print(f'Duration: {format_time(duration)}')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Final Portfolio Value: 10,000.00
Duration: 00:00:03

In [ ]:
log = pd.read_csv('backtest.csv')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
[Errno 2] No such file or directory: 'backtest.csv'
Traceback (most recent call last):
  File "/usr/local/lib64/python3.7/site-packages/pandas/io/parsers.py", line 610, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/usr/local/lib64/python3.7/site-packages/pandas/io/parsers.py", line 462, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/usr/local/lib64/python3.7/site-packages/pandas/io/parsers.py", line 819, in __init__
    self._engine = self._make_engine(self.engine)
  File "/usr/local/lib64/python3.7/site-packages/pandas/io/parsers.py", line 1050, in _make_engine
    return mapping[engine](self.f, **self.options)  # type: ignore[call-arg]
  File "/usr/local/lib64/python3.7/site-packages/pandas/io/parsers.py", line 1867, in __init__
    self._open_handles(src, kwds)
  File "/usr/local/lib64/python3.7/site-packages/pandas/io/parsers.py", line 1368, in _open_handles
    storage_options=kwds.get("storage_options", None),